In [ ]:
import pandas as pd
import numpy as np
import os

def allocate_load(time_stamp, method='weighted', usemat=True):
    """
    Allocate hourly load to the buses in NYS.
    
    Parameters:
        time_stamp: str, in "MM/dd/yyyy HH:mm:ss"
        method: str, "evenly" or "weighted"
        usemat: bool, if True uses .mat files otherwise .csv files

    Returns:
        sample_hour_load: DataFrame, load parameter table
    """
    
    # Read NYS bus table
    bus_info = import_bus_info(os.path.join("Data", "npcc_new.csv"))
    nys_bus = bus_info[bus_info['zone'] != 'NA']
    nys_bus_wload = nys_bus[nys_bus['sumLoadP0'] > 0]

    # Load bus and ratio calculation
    zone_ids = nys_bus['zone'].unique()
    load_bus_zone = [None] * 11
    load_ratio_zone = [None] * 11
    num_load_bus_zone = np.zeros(11, dtype=int)

    for i, zone_id in enumerate(zone_ids):
        load_bus_table = nys_bus_wload[nys_bus_wload['zone'] == zone_id]
        load_bus_zone[i] = load_bus_table['idx'].values
        if load_bus_table.shape[0] > 0:
            load_ratio_zone[i] = load_bus_table['sumLoadP0'] / load_bus_table['sumLoadP0'].sum()
        else:
            load_bus_zone[i] = nys_bus[nys_bus['zone'] == zone_id]['idx'].values
            load_ratio_zone[i] = np.ones(len(load_bus_zone[i])) / len(load_bus_zone[i])
        num_load_bus_zone[i] = len(load_bus_zone[i])
    
    num_load_bus_tot = sum(num_load_bus_zone)

    # Read load data
    if usemat:
        load_hourly = import_load(f'Data/loadHourly_{pd.to_datetime(time_stamp).year}.mat')
    else:
        load_hourly = import_load(f'Data/loadHourly_{pd.to_datetime(time_stamp).year}.csv')

    sample_load_zonal = load_hourly[load_hourly['TimeStamp'] == time_stamp]

    # Distribute load to buses
    if method == "evenly":
        sample_hour_load = add_load_evenly(sample_load_zonal, nys_bus, zone_ids)
    elif method == "weighted":
        sample_hour_load = add_load_weighted(sample_load_zonal, load_ratio_zone, load_bus_zone, num_load_bus_tot, nys_bus, zone_ids, num_load_bus_zone)
    else:
        raise ValueError("Error: Undefined load allocation method")
    
    return sample_hour_load

def add_load_evenly(hourly_load_zonal, nys_bus, zone_ids):
    nys_bus_load_even = pd.DataFrame()
    for zone_id in zone_ids:
        nys_bus_zone = nys_bus[nys_bus['zone'] == zone_id]
        zone_load_tot = hourly_load_zonal[hourly_load_zonal['ZoneID'] == zone_id]['Load'].values[0]
        nys_bus_zone['PD'] = zone_load_tot / len(nys_bus_zone)
        nys_bus_load_even = pd.concat([nys_bus_load_even, nys_bus_zone])

    power_factor = 0.98
    nys_bus_load_even['QD'] = nys_bus_load_even['PD'] * np.tan(np.arccos(power_factor))
    nys_bus_load_even = nys_bus_load_even[['idx', 'name', 'area', 'zone', 'PD', 'QD']]
    nys_bus_load_even = nys_bus_load_even.sort_values(by='idx')
    return nys_bus_load_even

def add_load_weighted(hourly_load_zonal, load_ratio_zone, load_bus_zone, num_load_bus_tot, nys_bus_old, zone_ids, num_load_bus_zone):
    zone_load_bus = [None] * 11
    load_bus_idx = np.zeros(num_load_bus_tot)
    load_bus_load = np.zeros(num_load_bus_tot)
    n = 0
    for i, zone_id in enumerate(zone_ids):
        zone_load_tot = hourly_load_zonal[hourly_load_zonal['ZoneID'] == zone_id]['Load'].values[0]
        zone_load_bus[i] = load_ratio_zone[i] * zone_load_tot
        load_bus_idx[n:n+num_load_bus_zone[i]] = load_bus_zone[i]
        load_bus_load[n:n+num_load_bus_zone[i]] = zone_load_bus[i]
        n += num_load_bus_zone[i]

    bus_wload = pd.DataFrame({'idx': load_bus_idx, 'PD': load_bus_load})
    bus_wload_cleaned = bus_wload.groupby('idx').sum().reset_index()

    # Join table
    nys_bus_new = pd.merge(nys_bus_old, bus_wload_cleaned, on='idx', how='left')
    nys_bus_new['PD'] = nys_bus_new['PD'].fillna(0)
    power_factor = 0.98
    nys_bus_new['QD'] = nys_bus_new['PD'] * np.tan(np.arccos(power_factor))
    return nys_bus_new

def import_bus_info(filename):
    """
    Import bus information from a CSV file.
    """
    bus_info = pd.read_csv(filename)
    return bus_info

def import_load(filename):
    """
    Import hourly load data in a year from a CSV file.
    """
    load = pd.read_csv(filename, parse_dates=['TimeStamp'], date_parser=lambda x: pd.to_datetime(x, format='%m/%d/%Y %H:%M:%S'))
    return load
